In [ ]:
# 10 Government Bus.
Bus_name_links = ["https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile",
                  "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile"]   

In [85]:
from selenium import webdriver                      
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # pip install webdriver_manager
driver.maximize_window()

driver.get("https://www.redbus.in/")# Driver will open a redbus page                  
time.sleep(2)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [87]:
                                                               #APSRTC Bus Routes & Timings

URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

In [91]:
# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

# Handle pagination
def handle_pagination(driver, wait):
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
        next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        time.sleep(1)

        try:
            next_page_button.click()
        except ElementNotInteractableException:
            driver.execute_script("arguments[0].click();", next_page_button)        
        print(f"Navigating to page {next_page_number}")
        time.sleep(10) 
        return True  
    
    except (NoSuchElementException, TimeoutException):
        print("No more pages to paginate or pagination element not found.")
        return False 

# scraping function
def main():
    driver = initialize_driver()
    wait = WebDriverWait(driver, 10)
    load_page(driver, URL)

    all_routes_name = []
    all_routes_link = []

    while True:
        # Scrape routes on the current page
        bus_routes_link, bus_routes_name = scrape_bus_routes(driver)
        all_routes_name.extend(bus_routes_name)
        all_routes_link.extend(bus_routes_link)
        
        # Handle pagination
        if not handle_pagination(driver, wait):
            break  
    # Save route data to CSV
    df_routes = pd.DataFrame({"Route_name": all_routes_name, "Route_link": all_routes_link})
    df_routes.to_csv("Apsrtc_bus_routes.csv", index=False)
    print("Data saved to bus_routes.csv")

    # scrape information for each route
    bus_details = []

    for i, row in df_routes.iterrows():
        link = row["Route_link"]
        routes = row["Route_name"]
        
        driver.get(link)
        time.sleep(2)

        # Click on all the route elements
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # click 'view buses' button if available
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except:
            continue
        time.sleep(2)

        # Scroll until page content stops updating        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5) 

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Collect bus details
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": routes,
                "Route_Link": link,
                "Bus_Name": bus_name[i].text if i < len(bus_name) else 'N/A',
                "Bus_Type": bus_type[i].text if i < len(bus_type) else 'N/A',
                "Departing_Time": departing_time[i].text if i < len(departing_time) else 'N/A',
                "Duration": total_duration[i].text if i < len(total_duration) else 'N/A',
                "Reaching_Time": reaching_time[i].text if i < len(reaching_time) else 'N/A',
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text if i < len(price) else 'N/A',
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        print(f"Successfully completed data extraction for route: {routes}")

    # Save detailed bus data to CSV
    df_buses = pd.DataFrame(bus_details)
    df_buses.to_csv("Apsrtc_bus_details.csv", index=False)
    print("Bus details saved to bus_details.csv")

    driver.quit()

if __name__ == "__main__":
    main()

Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found.
Data saved to bus_routes.csv
Successfully completed data extraction for route: Hyderabad to Ongole
Successfully completed data extraction for route: Ongole to Hyderabad
Successfully completed data extraction for route: Kakinada to Visakhapatnam
Successfully completed data extraction for route: Bangalore to Tirupati
Successfully completed data extraction for route: Bangalore to Kadapa
Successfully completed data extraction for route: Hyderabad to Narasaraopet
Successfully completed data extraction for route: Chittoor (Andhra Pradesh) to Bangalore
Successfully completed data extraction for route: Visakhapatnam to Kakinada
Successfully completed data extraction for route: Narasaraopet to Hyderabad
Successfully completed data extraction for route: Bangalore to Chittoor (Andhra Pradesh)
Successfully completed data extraction for route: Kadapa to Bang

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [13]:
                                                        # KSRTC(Kerala) Bus Routes & Timings
URL ="https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"

# Initialize WebDriver
def initialize_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

# Load the page
def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

In [15]:
# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

# Handle pagination
def handle_pagination(driver, wait):
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
        next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        time.sleep(1)
        
        try:
            next_page_button.click()
        except ElementNotInteractableException:
            driver.execute_script("arguments[0].click();", next_page_button)        
        print(f"Navigating to page {next_page_number}")
        time.sleep(10) 
        return True 

    except (NoSuchElementException, TimeoutException):
        print("No more pages to paginate or pagination element not found.")
        return False 

# scraping function
def main():
    driver = initialize_driver()
    wait = WebDriverWait(driver, 10)
    load_page(driver, URL)

    all_routes_name = []
    all_routes_link = []

    while True:
        # Scrape routes on the current page
        bus_routes_link, bus_routes_name = scrape_bus_routes(driver)
        all_routes_name.extend(bus_routes_name)
        all_routes_link.extend(bus_routes_link)
        
        # Handle pagination
        if not handle_pagination(driver, wait):
            break 

    # Save route data to CSV
    df_routes = pd.DataFrame({"Route_name": all_routes_name, "Route_link": all_routes_link})
    df_routes.to_csv("ksrtc_bus_routes.csv", index=False)
    print("Data saved to bus_routes.csv")

    # scrape information for each route
    bus_details = []

    for i, row in df_routes.iterrows():
        link = row["Route_link"]
        routes = row["Route_name"]
        
        driver.get(link)
        time.sleep(2)
        
        # Click on all the route elements
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # click 'view buses' button if available
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except:
            continue
        time.sleep(2)

        # Scroll until page content stops updating        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Collect bus details
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": routes,
                "Route_Link": link,
                "Bus_Name": bus_name[i].text if i < len(bus_name) else 'N/A',
                "Bus_Type": bus_type[i].text if i < len(bus_type) else 'N/A',
                "Departing_Time": departing_time[i].text if i < len(departing_time) else 'N/A',
                "Duration": total_duration[i].text if i < len(total_duration) else 'N/A',
                "Reaching_Time": reaching_time[i].text if i < len(reaching_time) else 'N/A',
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text if i < len(price) else 'N/A',
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        print(f"Successfully completed data extraction for route: {routes}")

    # Save detailed bus data to CSV
    df_buses = pd.DataFrame(bus_details)
    df_buses.to_csv("ksrtc_bus_details.csv", index=False)
    print("Bus details saved to bus_details.csv")

    driver.quit()

if __name__ == "__main__":
    main()

Navigating to page 2
No more pages to paginate or pagination element not found.
Data saved to bus_routes.csv
Successfully completed data extraction for route: Bangalore to Kozhikode
Successfully completed data extraction for route: Kozhikode to Ernakulam
Successfully completed data extraction for route: Ernakulam to Kozhikode
Successfully completed data extraction for route: Mysore to Kozhikode
Successfully completed data extraction for route: Bangalore to Kannur (Kerala)
Successfully completed data extraction for route: Kozhikode to Mysore
Successfully completed data extraction for route: Bangalore to Kalpetta (kerala)
Successfully completed data extraction for route: Kalpetta (kerala) to Bangalore
Successfully completed data extraction for route: Kannur (Kerala) to Bangalore
Successfully completed data extraction for route: Kozhikode to Thiruvananthapuram
Successfully completed data extraction for route: Kozhikode to Thrissur
Successfully completed data extraction for route: Thiruvan

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [19]:
                                                            #TGSRTC Bus Routes & Timings
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)
    

In [21]:
# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

# Handle pagination
def handle_pagination(driver, wait):
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
        next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        time.sleep(1)
        
        try:
            next_page_button.click()
        except ElementNotInteractableException:
            driver.execute_script("arguments[0].click();", next_page_button)        
        print(f"Navigating to page {next_page_number}")
        time.sleep(10) 
        return True

    except (NoSuchElementException, TimeoutException):
        print("No more pages to paginate or pagination element not found.")
        return False 

# scraping function
def main():
    driver = initialize_driver()
    wait = WebDriverWait(driver, 10)
    load_page(driver, URL)

    all_routes_name = []
    all_routes_link = []

    while True:
        # Scrape routes on the current page
        bus_routes_link, bus_routes_name = scrape_bus_routes(driver)
        all_routes_name.extend(bus_routes_name)
        all_routes_link.extend(bus_routes_link)
        
        # Handle pagination
        if not handle_pagination(driver, wait):
            break 

    # Save route data to CSV
    df_routes = pd.DataFrame({"Route_name": all_routes_name, "Route_link": all_routes_link})
    df_routes.to_csv("Tsrtc_bus_routes.csv", index=False)
    print("Data saved to bus_routes.csv")
    
    # scrape information for each route
    bus_details = []

    for i, row in df_routes.iterrows():
        link = row["Route_link"]
        routes = row["Route_name"]
        
        driver.get(link)
        time.sleep(2)
        
        # Click on all the route elements
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # click 'view buses' button if available
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except:
            continue
        time.sleep(2)

        # Scroll until page content stops updating        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5) 

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")  

        # Collect bus details
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": routes,
                "Route_Link": link,
                "Bus_Name": bus_name[i].text if i < len(bus_name) else 'N/A',
                "Bus_Type": bus_type[i].text if i < len(bus_type) else 'N/A',
                "Departing_Time": departing_time[i].text if i < len(departing_time) else 'N/A',
                "Duration": total_duration[i].text if i < len(total_duration) else 'N/A',
                "Reaching_Time": reaching_time[i].text if i < len(reaching_time) else 'N/A',
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text if i < len(price) else 'N/A',
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        print(f"Successfully completed data extraction for route: {routes}")

    # Save detailed bus data to CSV
    df_buses = pd.DataFrame(bus_details)
    df_buses.to_csv("Tsrtc_bus_details.csv", index=False)
    print("Bus details saved to bus_details.csv")

    driver.quit()

if __name__ == "__main__":
    main()

Navigating to page 2
Navigating to page 3
No more pages to paginate or pagination element not found.
Data saved to bus_routes.csv
Successfully completed data extraction for route: Hyderabad to Vijayawada
Successfully completed data extraction for route: Khammam to Hyderabad
Successfully completed data extraction for route: Hyderabad to Khammam
Successfully completed data extraction for route: Hyderabad to Mancherial
Successfully completed data extraction for route: Karimnagar to Hyderabad
Successfully completed data extraction for route: Hyderabad to Guntur (Andhra Pradesh)
Successfully completed data extraction for route: Guntur (Andhra Pradesh) to Hyderabad
Successfully completed data extraction for route: Hyderabad to Karimnagar
Successfully completed data extraction for route: Hyderabad to Ongole
Successfully completed data extraction for route: Hyderabad to Adilabad
Successfully completed data extraction for route: Hyderabad to Visakhapatnam
Successfully completed data extraction 

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [27]:
                                            #Kadamba Transport Corporation Limited (KTCL) Bus Routes & Timings
URL = "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5) 

In [29]:
# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

# Handle pagination
def handle_pagination(driver, wait):
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
        next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        time.sleep(1)
        
        try:
            next_page_button.click()
        except ElementNotInteractableException:
            driver.execute_script("arguments[0].click();", next_page_button)        
        print(f"Navigating to page {next_page_number}")
        time.sleep(10) 
        return True  
    
    except (NoSuchElementException, TimeoutException):
        print("No more pages to paginate or pagination element not found.")
        return False

# scraping function
def main():
    driver = initialize_driver()
    wait = WebDriverWait(driver, 10)
    load_page(driver, URL)

    all_routes_name = []
    all_routes_link = []

    while True:
        # Scrape routes on the current page
        bus_routes_link, bus_routes_name = scrape_bus_routes(driver)
        all_routes_name.extend(bus_routes_name)
        all_routes_link.extend(bus_routes_link)
        
        # Handle pagination
        if not handle_pagination(driver, wait):
            break 
    # Save route data to CSV
    df_routes = pd.DataFrame({"Route_name": all_routes_name, "Route_link": all_routes_link})
    df_routes.to_csv("ktcl_bus_routes.csv", index=False)
    print("Data saved to bus_routes.csv")
    
    # scrape information for each route
    bus_details = []

    for i, row in df_routes.iterrows():
        link = row["Route_link"]
        routes = row["Route_name"]
        
        driver.get(link)
        time.sleep(2)
        
        # Click on all the route elements
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # click 'view buses' button if available
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except:
            
            continue
            
        time.sleep(2)

        # Scroll until page content stops updating        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5) 

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]") 

        # Collect bus details
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": routes,
                "Route_Link": link,
                "Bus_Name": bus_name[i].text if i < len(bus_name) else 'N/A',
                "Bus_Type": bus_type[i].text if i < len(bus_type) else 'N/A',
                "Departing_Time": departing_time[i].text if i < len(departing_time) else 'N/A',
                "Duration": total_duration[i].text if i < len(total_duration) else 'N/A',
                "Reaching_Time": reaching_time[i].text if i < len(reaching_time) else 'N/A',
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text if i < len(price) else 'N/A',
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        print(f"Successfully completed data extraction for route: {routes}")

    # Save detailed bus data to CSV
    df_buses = pd.DataFrame(bus_details)
    df_buses.to_csv("ktcl_bus_details.csv", index=False)
    print("Bus details saved to bus_details.csv")

    driver.quit()

if __name__ == "__main__":
    main()

Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found.
Data saved to bus_routes.csv
Successfully completed data extraction for route: Pune to Goa
Bus details saved to bus_details.csv


In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [37]:
                                                         #RSRTC Bus Routes & Timings
URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

In [39]:
# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

# Handle pagination
def handle_pagination(driver, wait):
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
        next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        time.sleep(1)
        
        try:
            next_page_button.click()
        except ElementNotInteractableException:
            driver.execute_script("arguments[0].click();", next_page_button)        
        print(f"Navigating to page {next_page_number}")
        time.sleep(10) 
        return True  
    
    except (NoSuchElementException, TimeoutException):
        print("No more pages to paginate or pagination element not found.")
        return False

# scraping function
def main():
    driver = initialize_driver()
    wait = WebDriverWait(driver, 10)
    load_page(driver, URL)

    all_routes_name = []
    all_routes_link = []

    while True:
        # Scrape routes on the current page
        bus_routes_link, bus_routes_name = scrape_bus_routes(driver)
        all_routes_name.extend(bus_routes_name)
        all_routes_link.extend(bus_routes_link)
        
        # Handle pagination
        if not handle_pagination(driver, wait):
            break

    # Save route data to CSV
    df_routes = pd.DataFrame({"Route_name": all_routes_name, "Route_link": all_routes_link})
    df_routes.to_csv("Rsrtc_bus_routes.csv", index=False)
    print("Data saved to bus_routes.csv")
    
    # scrape information for each route
    bus_details = []

    for i, row in df_routes.iterrows():
        link = row["Route_link"]
        routes = row["Route_name"]
        
        driver.get(link)
        time.sleep(2)
        
        # Click on all the route elements
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # click 'view buses' button if available
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except:
            continue
        time.sleep(2)

        # Scroll until page content stops updating        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5) 

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")    

        # Collect bus details
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": routes,
                "Route_Link": link,
                "Bus_Name": bus_name[i].text if i < len(bus_name) else 'N/A',
                "Bus_Type": bus_type[i].text if i < len(bus_type) else 'N/A',
                "Departing_Time": departing_time[i].text if i < len(departing_time) else 'N/A',
                "Duration": total_duration[i].text if i < len(total_duration) else 'N/A',
                "Reaching_Time": reaching_time[i].text if i < len(reaching_time) else 'N/A',
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text if i < len(price) else 'N/A',
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        print(f"Successfully completed data extraction for route: {routes}")

    # Save detailed bus data to CSV
    df_buses = pd.DataFrame(bus_details)
    df_buses.to_csv("Rsrtc_bus_details.csv", index=False)
    print("Bus details saved to bus_details.csv")

    driver.quit()

if __name__ == "__main__":
    main()

Navigating to page 2
No more pages to paginate or pagination element not found.
Data saved to bus_routes.csv
Successfully completed data extraction for route: Jodhpur to Ajmer
Successfully completed data extraction for route: Jaipur (Rajasthan) to Jodhpur
Successfully completed data extraction for route: Beawar (Rajasthan) to Jaipur (Rajasthan)
Successfully completed data extraction for route: Udaipur to Jodhpur
Successfully completed data extraction for route: Sikar to Jaipur (Rajasthan)
Successfully completed data extraction for route: Aligarh (uttar pradesh) to Jaipur (Rajasthan)
Successfully completed data extraction for route: Kishangarh to Jaipur (Rajasthan)
Successfully completed data extraction for route: Jaipur (Rajasthan) to Aligarh (uttar pradesh)
Successfully completed data extraction for route: Jodhpur to Beawar (Rajasthan)
Successfully completed data extraction for route: Jaipur (Rajasthan) to Kota(Rajasthan)
Successfully completed data extraction for route: Udaipur to Pa

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [98]:
                                        #South Bengal State Transport Corporation (SBSTC) Bus Routes & Timings
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  


In [100]:
# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

# Handle pagination
def handle_pagination(driver, wait):
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
        next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        time.sleep(1)
        
        try:
            next_page_button.click()
        except ElementNotInteractableException:
            driver.execute_script("arguments[0].click();", next_page_button)        
        print(f"Navigating to page {next_page_number}")
        time.sleep(10) 
        return True  
    
    except (NoSuchElementException, TimeoutException):
        print("No more pages to paginate or pagination element not found.")
        return False  

# scraping function
def main():
    driver = initialize_driver()
    wait = WebDriverWait(driver, 10)
    load_page(driver, URL)

    all_routes_name = []
    all_routes_link = []

    while True:
        # Scrape routes on the current page
        bus_routes_link, bus_routes_name = scrape_bus_routes(driver)
        all_routes_name.extend(bus_routes_name)
        all_routes_link.extend(bus_routes_link)
        
        # Handle pagination
        if not handle_pagination(driver, wait):
            break

    # Save route data to CSV
    df_routes = pd.DataFrame({"Route_name": all_routes_name, "Route_link": all_routes_link})
    df_routes.to_csv("sbsrtc_bus_routes.csv", index=False)
    print("Data saved to bus_routes.csv")

    # scrape information for each route
    bus_details = []

    for i, row in df_routes.iterrows():
        link = row["Route_link"]
        routes = row["Route_name"]
        
        driver.get(link)
        time.sleep(2)
        
        # Click on all the route elements
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # click 'view buses' button if available
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except:
            continue
        time.sleep(2)

        # Scroll until page content stops updating        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5) 

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")  

        # Collect bus details
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": routes,
                "Route_Link": link,
                "Bus_Name": bus_name[i].text if i < len(bus_name) else 'N/A',
                "Bus_Type": bus_type[i].text if i < len(bus_type) else 'N/A',
                "Departing_Time": departing_time[i].text if i < len(departing_time) else 'N/A',
                "Duration": total_duration[i].text if i < len(total_duration) else 'N/A',
                "Reaching_Time": reaching_time[i].text if i < len(reaching_time) else 'N/A',
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text if i < len(price) else 'N/A',
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        print(f"Successfully completed data extraction for route: {routes}")

    # Save detailed bus data to CSV
    df_buses = pd.DataFrame(bus_details)
    df_buses.to_csv("Sbsrtc_bus_details.csv", index=False)
    print("Bus details saved to bus_details.csv")

    driver.quit()

if __name__ == "__main__":
    main()
    

Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found.
Data saved to bus_routes.csv
Successfully completed data extraction for route: Haldia to Kolkata
Successfully completed data extraction for route: Midnapore to Kolkata
Successfully completed data extraction for route: Kolkata to Midnapore
Successfully completed data extraction for route: Jhargram to Kolkata
Successfully completed data extraction for route: Kolkata to Contai (Kanthi)
Successfully completed data extraction for route: Kolkata to Nandakumar (west bengal)
Successfully completed data extraction for route: Durgapur (West Bengal) to Digha
Successfully completed data extraction for route: Kolkata to Suri
Successfully completed data extraction for route: Kolkata to Siliguri
Successfully completed data extraction for route: Siliguri to Kolkata
Successfully completed data extraction for route: Illambazar to Kolkata
Successfully completed da

In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [49]:
                                                              #HRTC Bus Routes & Timings
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

In [51]:
# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

# Handle pagination
def handle_pagination(driver, wait):
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
        next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        time.sleep(1)
        
        try:
            next_page_button.click()
        except ElementNotInteractableException:
            driver.execute_script("arguments[0].click();", next_page_button)        
        print(f"Navigating to page {next_page_number}")
        time.sleep(10) 
        return True  
    
    except (NoSuchElementException, TimeoutException):
        print("No more pages to paginate or pagination element not found.")
        return False 

# scraping function
def main():
    driver = initialize_driver()
    wait = WebDriverWait(driver, 10)
    load_page(driver, URL)

    all_routes_name = []
    all_routes_link = []

    while True:
        # Scrape routes on the current page
        bus_routes_link, bus_routes_name = scrape_bus_routes(driver)
        all_routes_name.extend(bus_routes_name)
        all_routes_link.extend(bus_routes_link)
        
        # Handle pagination
        if not handle_pagination(driver, wait):
            break 

    # Save route data to CSV
    df_routes = pd.DataFrame({"Route_name": all_routes_name, "Route_link": all_routes_link})
    df_routes.to_csv("Hrtc_bus_routes.csv", index=False)
    print("Data saved to bus_routes.csv")
    
    # scrape information for each route
    bus_details = []

    for i, row in df_routes.iterrows():
        link = row["Route_link"]
        routes = row["Route_name"]
        
        driver.get(link)
        time.sleep(2)
        
        # Click on all the route elements
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # click 'view buses' button if available
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except:
            continue
        time.sleep(2)

        # Scroll until page content stops updating        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)        

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Collect bus details
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": routes,
                "Route_Link": link,
                "Bus_Name": bus_name[i].text if i < len(bus_name) else 'N/A',
                "Bus_Type": bus_type[i].text if i < len(bus_type) else 'N/A',
                "Departing_Time": departing_time[i].text if i < len(departing_time) else 'N/A',
                "Duration": total_duration[i].text if i < len(total_duration) else 'N/A',
                "Reaching_Time": reaching_time[i].text if i < len(reaching_time) else 'N/A',
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text if i < len(price) else 'N/A',
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        print(f"Successfully completed data extraction for route: {routes}")

    # Save detailed bus data to CSV
    df_buses = pd.DataFrame(bus_details)
    df_buses.to_csv("Hrtc_bus_details.csv", index=False)
    print("Bus details saved to bus_details.csv")

    driver.quit()

if __name__ == "__main__":
    main()

Navigating to page 2
Navigating to page 3
Navigating to page 4
No more pages to paginate or pagination element not found.
Data saved to bus_routes.csv
Successfully completed data extraction for route: Delhi to Shimla
Successfully completed data extraction for route: Shimla to Delhi
Successfully completed data extraction for route: Chandigarh to Hamirpur (Himachal Pradesh)
Successfully completed data extraction for route: Hamirpur (Himachal Pradesh) to Chandigarh
Successfully completed data extraction for route: Shimla to Manali
Successfully completed data extraction for route: Delhi to Chandigarh
Successfully completed data extraction for route: Chandigarh to Manali
Successfully completed data extraction for route: Delhi to Manali
Successfully completed data extraction for route: Manali to Chandigarh
Successfully completed data extraction for route: Dharamshala (Himachal Pradesh) to Chandigarh
Successfully completed data extraction for route: Shimla to Chandigarh
Successfully completed

In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [59]:
                                                    #Assam State Transport Corporation (ASTC) OFFERS                                        
URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

In [61]:
# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

# Handle pagination
def handle_pagination(driver, wait):
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
        next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        time.sleep(1)
        
        try:
            next_page_button.click()
        except ElementNotInteractableException:
            driver.execute_script("arguments[0].click();", next_page_button)        
        print(f"Navigating to page {next_page_number}")
        time.sleep(10) 
        return True  
    
    except (NoSuchElementException, TimeoutException):
        print("No more pages to paginate or pagination element not found.")
        return False 

# scraping function
def main():
    driver = initialize_driver()
    wait = WebDriverWait(driver, 10)
    load_page(driver, URL)

    all_routes_name = []
    all_routes_link = []

    while True:
        # Scrape routes on the current page
        bus_routes_link, bus_routes_name = scrape_bus_routes(driver)
        all_routes_name.extend(bus_routes_name)
        all_routes_link.extend(bus_routes_link)
        
        # Handle pagination
        if not handle_pagination(driver, wait):
            break

    # Save route data to CSV
    df_routes = pd.DataFrame({"Route_name": all_routes_name, "Route_link": all_routes_link})
    df_routes.to_csv("Astc_bus_routes.csv", index=False)
    print("Data saved to bus_routes.csv")
    
    # scrape information for each route
    bus_details = []

    for i, row in df_routes.iterrows():
        link = row["Route_link"]
        routes = row["Route_name"]
        
        driver.get(link)
        time.sleep(2)
        
        # Click on all the route elements
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # click 'view buses' button if available
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except:
            continue
        time.sleep(2)

        # Scroll until page content stops updating        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)        

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Collect bus details
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": routes,
                "Route_Link": link,
                "Bus_Name": bus_name[i].text if i < len(bus_name) else 'N/A',
                "Bus_Type": bus_type[i].text if i < len(bus_type) else 'N/A',
                "Departing_Time": departing_time[i].text if i < len(departing_time) else 'N/A',
                "Duration": total_duration[i].text if i < len(total_duration) else 'N/A',
                "Reaching_Time": reaching_time[i].text if i < len(reaching_time) else 'N/A',
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text if i < len(price) else 'N/A',
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        print(f"Successfully completed data extraction for route: {routes}")

    # Save detailed bus data to CSV
    df_buses = pd.DataFrame(bus_details)
    df_buses.to_csv("Astc_bus_details.csv", index=False)
    print("Bus details saved to bus_details.csv")

    driver.quit()

if __name__ == "__main__":
    main()

Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found.
Data saved to bus_routes.csv
Successfully completed data extraction for route: Tezpur to Guwahati
Successfully completed data extraction for route: Guwahati to Tezpur
Successfully completed data extraction for route: Nagaon (Assam) to Guwahati
Successfully completed data extraction for route: Guwahati to Nagaon (Assam)
Successfully completed data extraction for route: Jorhat to Tinsukia
Successfully completed data extraction for route: Biswanath Charali to Guwahati
Successfully completed data extraction for route: North Lakhimpur to Tezpur
Successfully completed data extraction for route: North Lakhimpur to Dibrugarh
Successfully completed data extraction for route: Guwahati to Kaliabor
Successfully completed data extraction for route: Dibrugarh to North Lakhimpur
Successfully completed data extraction for route: North Lakhimpur to Nagaon (Assam

In [63]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [104]:
                                                        #WBTC (CTC) Bus Routes & Timings
URL = "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

In [106]:
# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

# Handle pagination
def handle_pagination(driver, wait):
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
        next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        time.sleep(1)
        
        try:
            next_page_button.click()
        except ElementNotInteractableException:
            driver.execute_script("arguments[0].click();", next_page_button)        
        print(f"Navigating to page {next_page_number}")
        time.sleep(10) 
        return True  
    
    except (NoSuchElementException, TimeoutException):
        print("No more pages to paginate or pagination element not found.")
        return False

# scraping function
def main():
    driver = initialize_driver()
    wait = WebDriverWait(driver, 10)
    load_page(driver, URL)

    all_routes_name = []
    all_routes_link = []

    while True:
        # Scrape routes on the current page
        bus_routes_link, bus_routes_name = scrape_bus_routes(driver)
        all_routes_name.extend(bus_routes_name)
        all_routes_link.extend(bus_routes_link)
        
        # Handle pagination
        if not handle_pagination(driver, wait):
            break  

    # Save route data to CSV
    df_routes = pd.DataFrame({"Route_name": all_routes_name, "Route_link": all_routes_link})
    df_routes.to_csv("Wbtc_bus_routes.csv", index=False)
    print("Data saved to bus_routes.csv")
    
    # scrape information for each route
    bus_details = []

    for i, row in df_routes.iterrows():
        link = row["Route_link"]
        routes = row["Route_name"]
        
        driver.get(link)
        time.sleep(2)
        
        # Click on all the route elements
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # click 'view buses' button if available
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except:
            continue
        time.sleep(2)

        # Scroll until page content stops updating        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)        

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")  

        # Collect bus details
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": routes,
                "Route_Link": link,
                "Bus_Name": bus_name[i].text if i < len(bus_name) else 'N/A',
                "Bus_Type": bus_type[i].text if i < len(bus_type) else 'N/A',
                "Departing_Time": departing_time[i].text if i < len(departing_time) else 'N/A',
                "Duration": total_duration[i].text if i < len(total_duration) else 'N/A',
                "Reaching_Time": reaching_time[i].text if i < len(reaching_time) else 'N/A',
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text if i < len(price) else 'N/A',
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        print(f"Successfully completed data extraction for route: {routes}")

    # Save detailed bus data to CSV
    df_buses = pd.DataFrame(bus_details)
    df_buses.to_csv("Wbtc_bus_details.csv", index=False)
    print("Bus details saved to bus_details.csv")

    driver.quit()

if __name__ == "__main__":
    main()

Navigating to page 2
Navigating to page 3
Navigating to page 4
No more pages to paginate or pagination element not found.
Data saved to bus_routes.csv
Successfully completed data extraction for route: Durgapur (West Bengal) to Kolkata
Successfully completed data extraction for route: Midnapore to Kolkata
Successfully completed data extraction for route: Digha to Barasat (West Bengal)
Successfully completed data extraction for route: Kolkata to Digha
Successfully completed data extraction for route: Digha to Kolkata
Successfully completed data extraction for route: Barasat (West Bengal) to Kolaghat
Successfully completed data extraction for route: Barasat (West Bengal) to Nandakumar (west bengal)
Successfully completed data extraction for route: Barasat (West Bengal) to Contai (Kanthi)
Successfully completed data extraction for route: Haldia to Kolkata
Successfully completed data extraction for route: Kolkata to Siliguri
Successfully completed data extraction for route: Barasat (West Be

In [71]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [114]:
                                                               #UPSRTC Bus Routes & Timings
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

In [116]:
# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

# Handle pagination
def handle_pagination(driver, wait):
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
        next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        time.sleep(1)
        
        try:
            next_page_button.click()
        except ElementNotInteractableException:
            driver.execute_script("arguments[0].click();", next_page_button)        
        print(f"Navigating to page {next_page_number}")
        time.sleep(10) 
        return True  
    
    except (NoSuchElementException, TimeoutException):
        print("No more pages to paginate or pagination element not found.")
        return False  

# scraping function
def main():
    driver = initialize_driver()
    wait = WebDriverWait(driver, 10)
    load_page(driver, URL)

    all_routes_name = []
    all_routes_link = []

    while True:
        # Scrape routes on the current page
        bus_routes_link, bus_routes_name = scrape_bus_routes(driver)
        all_routes_name.extend(bus_routes_name)
        all_routes_link.extend(bus_routes_link)
        
        # Handle pagination
        if not handle_pagination(driver, wait):
            break

    # Save route data to CSV
    df_routes = pd.DataFrame({"Route_name": all_routes_name, "Route_link": all_routes_link})
    df_routes.to_csv("Upsrtc_bus_routes.csv", index=False)
    print("Data saved to bus_routes.csv")
    
    # scrape information for each route
    bus_details = []

    for i, row in df_routes.iterrows():
        link = row["Route_link"]
        routes = row["Route_name"]
        
        driver.get(link)
        time.sleep(2)

        # Click on all the route elements
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # click 'view buses' button if available
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except:
            continue
        time.sleep(2)

        # Scroll until page content stops updating        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")        
        
        # Collect bus details
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": routes,
                "Route_Link": link,
                "Bus_Name": bus_name[i].text if i < len(bus_name) else 'N/A',
                "Bus_Type": bus_type[i].text if i < len(bus_type) else 'N/A',
                "Departing_Time": departing_time[i].text if i < len(departing_time) else 'N/A',
                "Duration": total_duration[i].text if i < len(total_duration) else 'N/A',
                "Reaching_Time": reaching_time[i].text if i < len(reaching_time) else 'N/A',
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text if i < len(price) else 'N/A',
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        print(f"Successfully completed data extraction for route: {routes}")

    # Save detailed bus data to CSV
    df_buses = pd.DataFrame(bus_details)
    df_buses.to_csv("Upsrtc_bus_details.csv", index=False)
    print("Bus details saved to bus_details.csv")

    driver.quit()

if __name__ == "__main__":
    main()

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (634, 2103)
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x010CEC13+23731]
	(No symbol) [0x0105C394]
	(No symbol) [0x00F3BE63]
	(No symbol) [0x00F85C97]
	(No symbol) [0x00F840E9]
	(No symbol) [0x00F81D0D]
	(No symbol) [0x00F81022]
	(No symbol) [0x00F75F57]
	(No symbol) [0x00FA1E5C]
	(No symbol) [0x00F759A4]
	(No symbol) [0x00FA20F4]
	(No symbol) [0x00FBB46E]
	(No symbol) [0x00FA1BF6]
	(No symbol) [0x00F73F35]
	(No symbol) [0x00F74EBD]
	GetHandleVerifier [0x013AF0D3+3039603]
	GetHandleVerifier [0x013C2DEA+3120778]
	GetHandleVerifier [0x013BB592+3089970]
	GetHandleVerifier [0x011643B0+635984]
	(No symbol) [0x01064DCD]
	(No symbol) [0x01062068]
	(No symbol) [0x01062205]
	(No symbol) [0x01054FD0]
	BaseThreadInitThunk [0x77557BA9+25]
	RtlInitializeExceptionChain [0x7794C0CB+107]
	RtlClearBits [0x7794C04F+191]


In [77]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [79]:
                                               #Bihar state road transport corporation (BSRTC)
URL = "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

In [81]:
# Scrape bus routes
def scrape_bus_routes(driver):
    try:
        route_elements = driver.find_elements(By.CLASS_NAME, 'route')
        bus_routes_link = [route.get_attribute('href') for route in route_elements]
        bus_routes_name = [route.text.strip() for route in route_elements]
        return bus_routes_link, bus_routes_name
    except Exception as e:
        print(f"Error scraping bus routes: {e}")
        return [], []

# Handle pagination
def handle_pagination(driver, wait):
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
        next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
        time.sleep(1)
        
        try:
            next_page_button.click()
        except ElementNotInteractableException:
            driver.execute_script("arguments[0].click();", next_page_button)        
        print(f"Navigating to page {next_page_number}")
        time.sleep(10) 
        return True  
    
    except (NoSuchElementException, TimeoutException):
        print("No more pages to paginate or pagination element not found.")
        return False 

# scraping function
def main():
    driver = initialize_driver()
    wait = WebDriverWait(driver, 10)
    load_page(driver, URL)

    all_routes_name = []
    all_routes_link = []

    while True:
        # Scrape routes on the current page
        bus_routes_link, bus_routes_name = scrape_bus_routes(driver)
        all_routes_name.extend(bus_routes_name)
        all_routes_link.extend(bus_routes_link)
        
        # Handle pagination
        if not handle_pagination(driver, wait):
            break 

    # Save route data to CSV
    df_routes = pd.DataFrame({"Route_name": all_routes_name, "Route_link": all_routes_link})
    df_routes.to_csv("Bsrtc_bus_routes.csv", index=False)
    print("Data saved to bus_routes.csv")
    
    # scrape information for each route
    bus_details = []

    for i, row in df_routes.iterrows():
        link = row["Route_link"]
        routes = row["Route_name"]
        
        driver.get(link)
        time.sleep(2)
        
        # Click on all the route elements
        elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
        for element in elements:
            element.click()
            time.sleep(2)

        # click 'view buses' button if available
        try:
            clicks = driver.find_element(By.XPATH, "//div[@class='button']")
            clicks.click()
        except:
            continue
        time.sleep(2)

        # Scroll until page content stops updating        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)        

        # Extract bus details
        bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        departing_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        reaching_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
        seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]") 

        # Collect bus details
        for i in range(len(bus_name)):
            bus_detail = {
                "Route_Name": routes,
                "Route_Link": link,
                "Bus_Name": bus_name[i].text if i < len(bus_name) else 'N/A',
                "Bus_Type": bus_type[i].text if i < len(bus_type) else 'N/A',
                "Departing_Time": departing_time[i].text if i < len(departing_time) else 'N/A',
                "Duration": total_duration[i].text if i < len(total_duration) else 'N/A',
                "Reaching_Time": reaching_time[i].text if i < len(reaching_time) else 'N/A',
                "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                "Price": price[i].text if i < len(price) else 'N/A',
                "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
            }
            bus_details.append(bus_detail)
        print(f"Successfully completed data extraction for route: {routes}")

    # Save detailed bus data to CSV
    df_buses = pd.DataFrame(bus_details)
    df_buses.to_csv("Bsrtc_bus_details.csv", index=False)
    print("Bus details saved to bus_details.csv")

    driver.quit()

if __name__ == "__main__":
    main()

Navigating to page 2
Navigating to page 3
Navigating to page 4
No more pages to paginate or pagination element not found.
Data saved to bus_routes.csv
Successfully completed data extraction for route: Patna (Bihar) to Motihari
Successfully completed data extraction for route: Patna (Bihar) to Bettiah
Successfully completed data extraction for route: Bettiah to Patna (Bihar)
Successfully completed data extraction for route: Patna (Bihar) to Balmiki Nagar (bihar)
Successfully completed data extraction for route: Balmiki Nagar (bihar) to Patna (Bihar)
Successfully completed data extraction for route: Patna (Bihar) to Purnea
Successfully completed data extraction for route: Patna (Bihar) to Darbhanga
Successfully completed data extraction for route: Patna (Bihar) to Muzaffarpur (Bihar)
Successfully completed data extraction for route: Patna (Bihar) to Chhapra (Bihar)
Bus details saved to bus_details.csv
